
### Maintenance dredging due to keeping the port accessible

![Alt text](image-1.png)

* This notebook aims to formulate the port accessibility in openclsim simulation when addressing the interaction between one dredging vessel and one seagoing vessel.

* The concept of accessibility is defined based on the picture above. When the available water depth is higher than the required water depth for the seagoing vessel, the vessel can sail through the port. Otherwise, the dredging vessel should start its job and keep the port accessible by dredging a certain amount of sediment.

The following variables are used:

```
AWL: Actual (as in at this moment) Water Level (meters)
LAT: Lowest Astronomical Tide (meters)
MBL: Maintained Bed Level (meters)
T: Draught (meters)
```
Then the criteria for accessibility can be defined as follows.

* required water depth = ```T``` + safety margin for under keel clearance

* available water depth = ```AWL - MBL```

Considerations

*  A dredging criteria level is defined to determine a level that when reaches the ```MBL```, the dredging operation is triggered. Dredging criteria level is considered as a safety margin to guarantee the nautical depth. When this safety margin is lower than the ```MBL```, no dredging is needed and the location is accessible. Otherwise, dredging is triggered.

* Dredging criteria level depends on errors in surveys and uncertainties in dredging caused by different rates of sedimentation. 

* Dredging criteria level = ```MBL``` - errors in surveys - uncertainties in dredging (dredging tolerance)

* To dredge navigation channels, ```LAT``` can be ignored because navigation channels are designed based on ```LAT``` and no vessel has tidal windows. When sailing through channels, vessels sail during the neap tide and not tidal restriction is applied.

* For berth location, ```LAT``` is considered as one of the determining factors when the minimum water level is a bit lower than ```LAT```. In this case, vessels need to be at the port during the ebb tide. Dredging criteria level are also taken into account to determine how frequently the berth should be dredged.

#### 0. Import libraries

In [1]:
import datetime

import networkx as nx
import shapely
import shapely.geometry
import pyproj


import datetime as dt
import time
import simpy
import openclsim.core as core
import openclsim.model as model
import openclsim.plugins as plugin
import openclsim.plot as plot
import opentnsim

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import folium

#### 1. Initialise simpy environment

In [2]:
# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes

In [3]:
# create a Site object based on desired mixin classes
DredgedSite = type(
    "DredgedSite",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
        core.HasDredging,
    ),
    {},
)

Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.ContainerDependentMovable,
        core.HasResource,
        core.HasContainer,
        core.Processor,
        core.Identifiable,
        core.Log,
        core.LoadingFunction,
        core.UnloadingFunction,
        # capture extra metadata to make sure we don't have leftover arguments
        opentnsim.core.ExtraMetadata,
    ),
    {},
)

#### 3. Create objects

##### 3.1. Create site objects

In [4]:
# prepare input data for offshore
location_offshore = shapely.geometry.Point(3.637610458072877, 52.10701582514347)
data_offshore = {
    "env": my_env,
    "name": "offshore",
    "geometry": location_offshore,
    "capacity": 100,
    "level": 0,
}
offshore = Site(**data_offshore)

# prepare input data for berth
location_berth = shapely.geometry.Point(4.062705498083785, 51.95909715255291)
data_berth = {
    "env": my_env,
    "name": "berth",
    "geometry": location_berth,
    "capacity": 100,
    "level": 80,
    "nr_resources": 1,
    # actual bed level is 0.5 above
    "ABL": -18,
    # dredging criterion level
    "DCL": -17.5,
    # dredge bed level (after dredging)
    "DBL": -18.5,
    # sedimentation rate of 1m per week, in m/s
    "SR": 1 / (3600 * 24 * 7)
    
}
berth = DredgedSite(**data_berth)

# prepare input data for dump
location_dump = shapely.geometry.Point(4.1795229742823, 52.11590591837503)
data_dump = {
    "env": my_env,
    "name": "dump",
    "geometry": location_dump,
    "capacity": 100,
    "level": 0,
}
dump = Site(**data_dump)

##### 3.2. Create vessel objects

In [5]:
# prepare input data for vessel_01 (dredging vessel)
data_vessel01 = {
    "env": my_env,
    "name": "dredging vessel",
    "geometry": location_dump,
    "capacity": 10,
    "loading_rate": 1,
    "unloading_rate": 0.5,
    "compute_v": lambda x: 10 + 2 * x,
}
# instantiate vessel_01
vessel01 = TransportProcessingResource(**data_vessel01)
assert not vessel01.metadata


# prepare input data for vessel_02 (seagoing vessel)
data_vessel02 = {
    "env": my_env,
    "name": "seagoing vessel",
    "geometry": location_offshore,
    "capacity": 10,
    "level": 5,
    "loading_rate": 1,
    "unloading_rate": 0.5,
    "compute_v": lambda x: 15 + 2 * x,
}
# instantiate vessel_02
vessel02 = TransportProcessingResource(**data_vessel02)
assert not vessel02.metadata

##### 3.3. Creat activity objects

In [6]:
# Initialise registry
registry = {}

In [7]:
# Define custom activities with Depth Plugin
MoveWhenDredgingNeeded = type(
    "MoveWhenDredgingNeeded",
    (
        plugin.HasDredgePluginActivity,
        model.MoveActivity,
    ),
    {},
)

In [8]:
# Specify the amount of container and duration of activities
amount = 10
duration = 200


dredge_criteria = plugin.DredgeCriterion(
    name="dredge_crit",
)

# Specify dredging subprocesses
dredging_subprocesses = [
    MoveWhenDredgingNeeded(
        env=my_env,
        name="sailing empty dredging vessel",
        registry=registry,
        mover=vessel01,
        destination=berth,
        dredge_criteria=dredge_criteria,
    ),
    model.ShiftAmountActivity(
        env=my_env,
        name="loading dredging vessel",
        registry=registry,
        processor=vessel01,
        origin=berth,
        destination=vessel01,
        amount=amount,
        duration=50 * amount,
    ),
    model.MoveActivity(
        env=my_env,
        name="sailing full dredging vessel",
        registry=registry,
        mover=vessel01,
        destination=dump,
    ),
    model.ShiftAmountActivity(
        env=my_env,
        name="unloading dredging vessel",
        registry=registry,
        processor=vessel01,
        origin=vessel01,
        destination=dump,
        amount=amount,
        duration=20 * amount,
    ),
]

# Define activities of the dredging vessel
activities_dredging_vessel = model.WhileActivity(
    env=my_env,
    name="while sequential activity dredging vessel",
    registry=registry,
    sub_processes=[
        model.SequentialActivity(
            env=my_env,
            name=f"sequential activity subcycle dredging vessel",
            registry=registry,
            sub_processes=dredging_subprocesses,
        )
    ],
    condition_event=[
        {
            "type": "container",
            "concept": berth,
            "state": "empty",
            "id_": "default_reservations",
        }
    ],
)

In [9]:
# Specify seagoing subprocesses
requested_resources = {}
seagoing_subprocesses = [
    model.BasicActivity(
        env=my_env, name="interval time", registry=registry, duration=200
    ),
    model.MoveActivity(
        env=my_env,
        name="sailing full seagoing vessel",
        registry=registry,
        mover=vessel02,
        destination=berth,
    ),
    model.ShiftAmountActivity(
        env=my_env,
        name="unloading seagoing vessel",
        registry=registry,
        processor=vessel02,
        origin=vessel02,
        destination=berth,
        amount=amount,
        duration=30 * amount,
        requested_resources=requested_resources,
    ),
    model.MoveActivity(
        env=my_env,
        name="sailing empty seagoing vessel",
        registry=registry,
        mover=vessel02,
        destination=offshore,
    ),
]

# Define activities of the seagoing vessel
activities_seagoing_vessel = model.WhileActivity(
    env=my_env,
    name="while sequential activity dredging vessel",
    registry=registry,
    sub_processes=seagoing_subprocesses,
    condition_event=[
        {
            "type": "container",
            "concept": vessel02,
            "state": "empty",
            "id_": "default_reservations",
        }
    ],
)

#### 4. Register activities and run simpy

In [10]:
# register dredging activities
model.register_processes([activities_dredging_vessel])

# register seagoing activities
model.register_processes([activities_seagoing_vessel])

In [11]:
my_env.run()

mover dredging vessel destination berth
check dredging criteria 0
wait until we need to dredge again
remaining_bed_level: 0.5
waiting for 302400.0 seconds
delaying processing <__main__.MoveWhenDredgingNeeded object at 0x139d2a910>
mover seagoing vessel destination berth
updating to destination geometry POINT (4.062705498083785 51.95909715255291)
mover seagoing vessel destination offshore
updating to destination geometry POINT (3.637610458072877 52.10701582514347)
updating to destination geometry POINT (4.062705498083785 51.95909715255291)
mover dredging vessel destination dump
updating to destination geometry POINT (4.1795229742823 52.11590591837503)
mover dredging vessel destination berth
check dredging criteria 306620.2072004737
wait until we need to dredge again
remaining_bed_level: 0.5
waiting for 302400.0 seconds
delaying processing <__main__.MoveWhenDredgingNeeded object at 0x139d2a910>
updating to destination geometry POINT (4.062705498083785 51.95909715255291)
mover dredging ve

In [12]:
# the dredging log is now not in the ship but in the plugin....
dredging_cycle_log_df = pd.DataFrame(activities_dredging_vessel.sub_processes[0].sub_processes[0].logbook)
dredging_cycle_log_df

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00.000000,36d84ac4-5a80-4ba0-9ee2-bae4e0d8309d,WAIT_START,{},{}
1,1970-01-01 00:00:00.000000,36d84ac4-5a80-4ba0-9ee2-bae4e0d8309d,WAIT_STOP,{},{}
2,1970-01-01 00:00:00.000000,36d84ac4-5a80-4ba0-9ee2-bae4e0d8309d,WAIT_START,{},"{'type': 'plugin', 'ref': 'dredging'}"
3,1970-01-04 12:00:00.000000,36d84ac4-5a80-4ba0-9ee2-bae4e0d8309d,WAIT_STOP,{},"{'type': 'plugin', 'ref': 'dredging'}"
4,1970-01-04 12:00:00.000000,36d84ac4-5a80-4ba0-9ee2-bae4e0d8309d,START,{},{}
5,1970-01-04 12:32:00.113018,36d84ac4-5a80-4ba0-9ee2-bae4e0d8309d,STOP,{},{}
6,1970-01-04 13:10:20.207200,36d84ac4-5a80-4ba0-9ee2-bae4e0d8309d,WAIT_START,{},{}
7,1970-01-04 13:10:20.207200,36d84ac4-5a80-4ba0-9ee2-bae4e0d8309d,WAIT_STOP,{},{}
8,1970-01-04 13:10:20.207200,36d84ac4-5a80-4ba0-9ee2-bae4e0d8309d,WAIT_START,{},"{'type': 'plugin', 'ref': 'dredging'}"
9,1970-01-08 01:10:20.207200,36d84ac4-5a80-4ba0-9ee2-bae4e0d8309d,WAIT_STOP,{},"{'type': 'plugin', 'ref': 'dredging'}"


#### 5. Inspect results

##### 5.1. Inspect logs

In [13]:
# Inspect dredging vessel log
display(plot.get_log_dataframe(vessel01, [activities_dredging_vessel]))

,Activity,Timestamp,ActivityState,geometry,container level
0,sailing empty dredging vessel,1970-01-04 12:00:00.000000,START,POINT (4.1795229742823 52.11590591837503),0.0
1,sailing empty dredging vessel,1970-01-04 12:32:00.113018,STOP,POINT (4.062705498083785 51.95909715255291),0.0
2,loading dredging vessel,1970-01-04 12:32:00.113018,START,POINT (4.062705498083785 51.95909715255291),0.0
3,loading dredging vessel,1970-01-04 12:40:20.113018,STOP,POINT (4.062705498083785 51.95909715255291),10.0
4,sailing full dredging vessel,1970-01-04 12:40:20.113018,START,POINT (4.062705498083785 51.95909715255291),10.0
...,...,...,...,...,...
67,loading dredging vessel,1970-02-01 22:03:01.770622,STOP,POINT (4.062705498083785 51.95909715255291),5.0
68,sailing full dredging vessel,1970-02-01 22:03:01.770622,START,POINT (4.062705498083785 51.95909715255291),5.0
69,sailing full dredging vessel,1970-02-01 22:32:07.327912,STOP,POINT (4.1795229742823 52.11590591837503),5.0
70,unloading dredging vessel,1970-02-01 22:32:07.327912,START,POINT (4.1795229742823 52.11590591837503),5.0


In [14]:
# Inspect seagoing vessel log
display(plot.get_log_dataframe(vessel02, [activities_seagoing_vessel]))

,Activity,Timestamp,ActivityState,geometry,container level
0,sailing full seagoing vessel,1970-01-01 00:03:20.000000,START,POINT (3.637610458072877 52.10701582514347),5
1,sailing full seagoing vessel,1970-01-01 00:38:13.474629,STOP,POINT (4.062705498083785 51.95909715255291),5
2,unloading seagoing vessel,1970-01-01 00:38:13.474629,START,POINT (4.062705498083785 51.95909715255291),5
3,unloading seagoing vessel,1970-01-01 00:43:13.474629,STOP,POINT (4.062705498083785 51.95909715255291),0
4,sailing empty seagoing vessel,1970-01-01 00:43:13.474629,START,POINT (4.062705498083785 51.95909715255291),0
5,sailing empty seagoing vessel,1970-01-01 01:20:26.514232,STOP,POINT (3.637610458072877 52.10701582514347),0


##### 5.2. Visualize gantt chart

In [15]:
res = plot.get_gantt_chart([vessel01, vessel02], y_scale="numbers", static=True, id_map=[activities_dredging_vessel, activities_seagoing_vessel])
data = res['data']
layout = res['layout']

data.append({
    "name": "sailing_crit",
    "x":[vessel01.log["Timestamp"][0].isoformat(), vessel01.log["Timestamp"][-1].isoformat()],
    "y":[sailing_crit.minimum,sailing_crit.minimum],
})
data.append({
    "name": "sailing_crit",
    "x":[vessel02.log["Timestamp"][0].isoformat(), vessel02.log["Timestamp"][-1].isoformat()],
    "y":[sailing_crit.minimum,sailing_crit.minimum],
})


data.append({
    "name": "Depth",
    "x":depth_df.index,
    "y":depth_df[sailing_crit.condition],
})


data.append({
    "name": "loading_crit",
    "x":[vessel01.log["Timestamp"][0].isoformat(), vessel01.log["Timestamp"][-1].isoformat()],
    "y":[loading_crit.minimum,loading_crit.minimum],
})
data.append({
    "name": "loading_crit",
    "x":[vessel02.log["Timestamp"][0].isoformat(), vessel02.log["Timestamp"][-1].isoformat()],
    "y":[loading_crit.minimum,loading_crit.minimum],
})

go.Figure(data=data, layout=layout)

NameError: name 'sailing_crit' is not defined

In [ ]:
plot.get_gantt_chart(
    [vessel01, vessel02],
    id_map=[activities_dredging_vessel, activities_seagoing_vessel],
)

##### 5.3. Visualize container volume developments

In [ ]:
fig = plot.get_step_chart([vessel01, berth, dump])

In [ ]:
fig = plot.get_step_chart([vessel02, berth, offshore])